In [46]:
sc

In [47]:
dfhtag1 = sqlContext.read.load('sample_clean.jsonl', format='json')
dfhtag1.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [107]:
dfhtag2 = sqlContext.read.load('stream_twt_may8.jsonl', format='json')
dfhtag2.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_

In [90]:
hashtag1 = dfhtag1.select('entities.hashtags.text','timestamp_ms', 'place.full_name' ).dropna()
hashtag1 = hashtag1.filter(sf.size('text')>0)
hashtag1 = hashtag1.select(sf.explode('text').alias('hashtags'), 'timestamp_ms', 'full_name')
hashtag1 = hashtag1.select(sf.lower(hashtag1.hashtags).alias('hashtags'), 'timestamp_ms', 'full_name')
hashtag1.show()

+--------------------+-------------+-------------------+
|            hashtags| timestamp_ms|          full_name|
+--------------------+-------------+-------------------+
|  whatatimetobealive|1493392013161|          Indio, CA|
|     stgermainenlaye|1493392016065|     Sacramento, CA|
|            humility|1493392019250|        Chicago, IL|
|          tmomdisney|1493392061402|         Texas, USA|
|                tmom|1493392061402|         Texas, USA|
|                tdad|1493392061402|         Texas, USA|
|            imtheone|1493392066618|   Toronto, Ontario|
|               bills|1493392067881|        Buffalo, NY|
|              autism|1493392068933|       Coronado, CA|
|          delusional|1493392069446|  Overland Park, KS|
|           taxreform|1493392069435|        Potomac, MD|
|         iengagebwyn|1493392074213|         Berwyn, IL|
|        mysteryskype|1493392074213|         Berwyn, IL|
|         freaknation|1493392114680|        Phoenix, AZ|
|didyouevenseethel...|149339211

In [91]:
hashtag1 = hashtag1.groupBy('hashtags').count().orderBy('count', ascending = False)
hashtag1.show(truncate=False, n=1000) #6475
#hashtag1.toPandas().to_csv('tophashtags/hashtag1.csv', encoding='utf-8', index=False)


+-------------------------+-----+
|hashtags                 |count|
+-------------------------+-----+
|job                      |1414 |
|hiring                   |1205 |
|careerarc                |445  |
|jobs                     |285  |
|retail                   |130  |
|veterans                 |127  |
|hospitality              |124  |
|nfldraft                 |91   |
|transportation           |90   |
|healthcare               |86   |
|nursing                  |78   |
|sales                    |69   |
|it                       |61   |
|nationalbeerday          |60   |
|fyrefestival             |58   |
|trump                    |52   |
|trumprussia              |51   |
|maga                     |47   |
|resist                   |46   |
|local                    |46   |
|ff                       |45   |
|customerservice          |41   |
|unsignedartist           |39   |
|scotus                   |37   |
|automotive               |36   |
|friday                   |33   |
|houston      

In [114]:
import pyspark.sql.functions as sf
hashtag2 = dfhtag2.select('place.full_name').dropna()
#hashtag2 = dfhtag2.select('text')
# hashtag2 = hashtag2.filter(sf.size('text')>0)
# hashtag2 = hashtag2.select(sf.explode('text').alias('hashtags'))
# hashtag2 = hashtag2.select(sf.lower(hashtag2.hashtags).alias('hashtags'))
hashtag2.show(truncate = False, n=1000)
#hashtag2.show()

+----------------------------+
|full_name                   |
+----------------------------+
|Georgia, USA                |
|Parkersburg, WV             |
|Virginia, USA               |
|California, USA             |
|Ohio, USA                   |
|Florida, USA                |
|Houston, TX                 |
|Houston, TX                 |
|Antioch, CA                 |
|Country Homes, WA           |
|Queens, NY                  |
|Dallas, TX                  |
|Providence, RI              |
|Florence, KY                |
|Austin, TX                  |
|Virginia Beach, VA          |
|Houston, TX                 |
|Kenosha, WI                 |
|Riverside, CA               |
|Las Vegas, NV               |
|Meriden, CT                 |
|Norfolk, VA                 |
|Naperville, IL              |
|Abilene, TX                 |
|Dale City, VA               |
|Maple Heights, OH           |
|Lubbock, TX                 |
|Lee's Summit, MO            |
|Oakwood, OH                 |
|Blasdel

In [115]:
hashtag2 = hashtag2.groupBy('full_name').count().orderBy('count', ascending = False)
hashtag2.show(truncate=False, n=10000) #6475
#hashtag2.toPandas().to_csv('tophashtags/hashtag2.csv', encoding='utf-8', index=False)

+---------------------------------------------------------------+-----+
|full_name                                                      |count|
+---------------------------------------------------------------+-----+
|Houston, TX                                                    |1898 |
|Los Angeles, CA                                                |1501 |
|Georgia, USA                                                   |1205 |
|Florida, USA                                                   |1035 |
|Manhattan, NY                                                  |844  |
|Chicago, IL                                                    |837  |
|Texas, USA                                                     |810  |
|Atlanta, GA                                                    |536  |
|Washington, DC                                                 |531  |
|New Orleans, LA                                                |500  |
|Dallas, TX                                                     

In [7]:
# toploc.write.csv('toplocations/toplocations.csv')
#toploc.write.format('csv').save('toplocations/toplocations.csv')
result1.toPandas().to_csv('tophashtags/tophashtags.csv', encoding='utf-8', index=False)
